**Sistema Asistente IA con Generación Multimodal para Documentación Técnica**


Este proyecto implementa un sistema asistente de IA que combina la generación de texto y de imágenes para crear documentación técnica enriquecida. Utilizando técnicas de Few-Shot Prompting para el procesamiento de texto y herramientas de generación de imágenes, el sistema puede generar explicaciones técnicas acompañadas de visualizaciones relevantes.

**3.1 Presentación del problema**

En el ámbito de la documentación técnica, existe una brecha significativa entre la generación de explicaciones textuales y su acompañamiento visual. Los desarrolladores y documentadores técnicos frecuentemente necesitan:

1. Generar explicaciones técnicas precisas
2. Crear diagramas y visualizaciones que acompañen estas explicaciones
3. Mantener consistencia entre texto e imágenes
4. Optimizar el tiempo dedicado a la creación de contenido

Este proceso manual es costoso en tiempo y recursos, y puede llevar a inconsistencias entre el texto y las visualizaciones.

**Lab**

1. Import openAI dependences (load with conda install openai)

In [13]:
import openai
from typing import List, Dict, Tuple
import json

2. Use the API Key from openAI page

In [19]:
#Add our API Key copied from openAI page
openai.api_key = "sk-lCSnTlt3jrXXcC-LREjVO7ot2TtB8TUfHbOvA7AZhdT3BlbkFJg6zG3yXeyH_z5ajPkBFzUz531VdQbfhLNzwrm2oF0A" #Example -> dw-bQyr9128F49VJRNKAnST32lbkFJ2Zv4qEhWo4R0Xh6yrh15

# Verificar que la API key está configurada
print(f"API Key configurada: {'*' * len(openai.api_key) if openai.api_key else 'No configurada'}")

API Key configurada: ***********************************************************************************************


3.1 Make a request 

In [20]:
class MultimodalAssistant:
    def __init__(self, api_key: str = None):
        if api_key:
            openai.api_key = api_key
            
        self.reference_docs = {
            "architecture": """
            La arquitectura del sistema se basa en microservicios, con componentes
            distribuidos que se comunican a través de APIs REST. El sistema utiliza
            contenedores Docker para el despliegue y Kubernetes para la orquestación.
            Los servicios son independientes y pueden escalarse horizontalmente según
            la demanda. La comunicación entre servicios se realiza mediante APIs REST
            y mensajería asíncrona para garantizar la eficiencia y resiliencia.
            """,
            
            "workflow": """
            El flujo de trabajo incluye etapas de desarrollo, pruebas automatizadas,
            integración continua y despliegue continuo (CI/CD). Cada cambio pasa por
            un pipeline automatizado antes de llegar a producción. Las etapas incluyen
            pruebas unitarias, integración, análisis de código y despliegue automatizado.
            """,
            
            "security": """
            La seguridad se implementa en múltiples capas, incluyendo autenticación
            JWT, encriptación end-to-end, y monitoreo continuo de vulnerabilidades.
            Se aplican políticas de acceso basadas en roles y se realiza auditoría
            constante de todas las operaciones críticas.
            """
        }
        
        self.system_prompt = """
        Eres un experto en arquitectura de software y documentación técnica.
        Genera respuestas completas y bien estructuradas, usando párrafos claros.
        No uses más de 4-5 párrafos y asegúrate de que cada respuesta sea autocontenida.
        No menciones que vas a mostrar un diagrama al final de la respuesta.
        """
        
        self.image_templates = {
            "architecture": """
            Create a professional technical diagram with these specifications:
            
            Type: Microservices Architecture Diagram
            
            Required Elements:
            - Multiple interconnected microservices (5-7 services)
            - API Gateway
            - Load Balancer
            - Docker containers
            - Kubernetes cluster representation
            - Database systems
            
            Style Guidelines:
            - Modern and clean design
            - Professional blue color scheme
            - Clear service boundaries
            - Directional arrows for data flow
            
            Layout:
            - Horizontal flow (left to right)
            - Clear component separation
            - Logical grouping of services
            
            Labels:
            - Service names
            - Technology indicators
            - Connection types (REST, async)
            
            Additional Details:
            - Show scaling capabilities
            - Include monitoring points
            - Security boundaries
            """,
            
            "workflow": """
            Create a professional CI/CD workflow diagram with these specifications:
            
            Type: CI/CD Pipeline Visualization
            
            Required Elements:
            - Source control (Git)
            - Build stages
            - Test environments
            - Quality gates
            - Deployment stages
            
            Style Guidelines:
            - Clean and professional
            - Process flow indicators
            - Status checkpoints
            
            Layout:
            - Vertical flow (top to bottom)
            - Pipeline stages clearly separated
            - Parallel processes where applicable
            
            Labels:
            - Stage names
            - Tools used
            - Time estimates
            """
        }

    def get_text_response(self, query: str, context_type: str) -> str:
        """
        Genera una respuesta textual usando la API de OpenAI con un mejor control
        de la longitud y estructura
        """
        context = self.reference_docs[context_type]
        
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": f"""
            Contexto: {context}
            
            Consulta: {query}
            
            Genera una respuesta técnica detallada basada únicamente en la información del contexto.
            La respuesta debe ser completa y bien estructurada.
            """}
        ]
        
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=messages,
                max_tokens=500,  # Aumentado para asegurar respuestas completas
                temperature=0.7,
                presence_penalty=0.6,  # Ayuda a mantener el foco
                frequency_penalty=0.3   # Evita repeticiones
            )
            
            return response.choices[0].message.content.strip()
            
        except Exception as e:
            print(f"Error en generación de respuesta: {e}")
            return "No se pudo generar una respuesta. Por favor, intenta nuevamente."

    def process_query(self, query: str) -> Dict:
        """
        Procesa una consulta y genera la documentación completa
        """
        # Determinar el tipo de contexto (simplificado por ahora)
        context_type = "architecture" if "arquitectura" in query.lower() else "workflow"
        
        # Obtener respuesta textual
        text_response = self.get_text_response(query, context_type)
        
        # Preparar resultado
        result = {
            "query": query,
            "context_used": context_type,
            "text_response": text_response
        }
        
        # Si se requiere diagrama, incluir el prompt para imagen
        if "diagrama" in query.lower() or "visual" in query.lower():
            result["image_prompt"] = self.image_templates[context_type]
            result["image_tool"] = "Nightcafe"
            
        return result

# Crear instancia del asistente (no necesitas pasar la API key porque ya está configurada globalmente)
assistant = MultimodalAssistant()

# Realizar una consulta
query = "Explica la arquitectura de microservicios y genera un diagrama"
result = assistant.process_query(query)

# Mostrar resultados
print("\nResultado del procesamiento:")
print(f"Consulta: {result['query']}")
print(f"Contexto usado: {result['context_used']}")
print(f"Respuesta textual: {result['text_response']}")
if "image_prompt" in result:
    print("\nPrompt para generación de imagen:")
    print(result['image_prompt'])
    print(f"\nHerramienta de imagen: {result['image_tool']}")

# Probemos otra consulta para el flujo de trabajo
query_workflow = "Describe el flujo de trabajo CI/CD y muestra una visualización"
result_workflow = assistant.process_query(query_workflow)

print("\n" + "="*50 + "\n")
print("Segunda consulta - Flujo de trabajo:")
print(f"Consulta: {result_workflow['query']}")
print(f"Contexto usado: {result_workflow['context_used']}")
print(f"Respuesta textual: {result_workflow['text_response']}")
if "image_prompt" in result_workflow:
    print("\nPrompt para generación de imagen:")
    print(result_workflow['image_prompt'])
    print(f"\nHerramienta de imagen: {result_workflow['image_tool']}")


Resultado del procesamiento:
Consulta: Explica la arquitectura de microservicios y genera un diagrama
Contexto usado: architecture
Respuesta textual: La arquitectura de microservicios es un enfoque de diseño de software donde una aplicación se divide en múltiples servicios pequeños e independientes, cada uno centrado en realizar una tarea específica. En este caso, los microservicios del sistema se comunican entre sí a través de APIs REST, lo que permite la interoperabilidad y el intercambio de datos de forma sencilla y eficiente. Esta comunicación basada en APIs REST facilita la escalabilidad y flexibilidad del sistema al permitir que cada servicio evolucione de manera independiente.

Para el despliegue de los microservicios, se utiliza Docker, una tecnología de contenedores que permite empaquetar una aplicación junto con todas sus dependencias en un contenedor ligero y portátil. Esto garantiza la consistencia del entorno de ejecución en diferentes etapas del ciclo de vida del softwar